<a href="https://colab.research.google.com/github/OliverGriffiths287/OliverGriffiths287.github.io/blob/main/python_code/salary_season_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json
import pandas as pd

In [ ]:
# Define the URL of the page containing the salary data
url = "https://www.capology.com/uk/premier-league/salaries/2022-2023"

# Send a GET request to the URL to retrieve the page's HTML content
req = requests.get(url)

# Extract the text content of the page (HTML source)
page_src = req.text

# Find the starting index of the average salary section in the HTML content
# The salary information is located within a specific element containing 'accounting.formatMoney("'
start_index = page_src.index('$(\'#salaries-avg\').html(accounting.formatMoney("')

# Slice the page source starting from the location of the average salary element
page_src = page_src[start_index:]

# Split the sliced HTML content by double quotes to isolate the salary value
page_src = page_src.split('"')

# Extract the average salary value (second item in the split list) and assign it to 'avg'
avg = page_src[1]

# Print or return the average salary value
avg

In [ ]:
# Extract the HTML content of the page as text
page_src = req.text

# Split the page source to isolate the JSON-like data within 'var data = ...'
# This step assumes there's only one occurrence of 'var data =' in the page source
page_src = page_src.split("var data =")[1]

# Further split the result to exclude any trailing characters after the JSON data
# We're aiming to end the data at the first occurrence of ";\n"
page_src = page_src.split(";\n")[0]

# Clean the extracted data by removing the outer square brackets to prepare it for parsing
page_src = page_src.replace("[", "").replace("]", "")

# Display the cleaned data
page_src

# Split the data into individual player records, using '{' as the delimiter
# Each entry in 'players' represents data for one player
players = page_src.split("{")

# Initialize an empty list to store each player's parsed data as dictionaries
players_data = []

# Loop through each player entry in 'players' to parse their information
for player_text in players:
    # Create an empty dictionary to store parsed fields for this player
    parsed_player_data = {}

    # Remove the trailing "}," from the player data for easier parsing
    player_text = player_text.replace("},", "")

    # Replace any double newlines with single newlines to standardize formatting
    player_text = player_text.replace("\n\n", "\n")
# Extract the HTML content of the page as text
page_src = req.text

# Split the page source to isolate the JSON-like data within 'var data = ...'
# This step assumes there's only one occurrence of 'var data =' in the page source
page_src = page_src.split("var data =")[1]

# Further split the result to exclude any trailing characters after the JSON data
# We're aiming to end the data at the first occurrence of ";\n"
page_src = page_src.split(";\n")[0]

# Clean the extracted data by removing the outer square brackets to prepare it for parsing
page_src = page_src.replace("[", "").replace("]", "")

# Display the cleaned data
page_src

# Split the data into individual player records, using '{' as the delimiter
# Each entry in 'players' represents data for one player
players = page_src.split("{")

# Initialize an empty list to store each player's parsed data as dictionaries
players_data = []

# Loop through each player entry in 'players' to parse their information
for player_text in players:
    # Create an empty dictionary to store parsed fields for this player
    parsed_player_data = {}

    # Remove the trailing "}," from the player data for easier parsing
    player_text = player_text.replace("},", "")
    # Split the player's data into individual fields based on newlines
    player_fields = player_text.split("\n")

    # Uncomment the following line to print each player's text if needed
    # print(player_text)

    # Loop through each field in the player's data to extract key-value pairs
    for player_field in player_fields:
        # Skip empty lines
        if len(player_field) == 0:
            continue
        # Extract name, URL, and flag image URL if the field contains 'name'
        elif 'name' in player_field:
            # Get the link
            parsed_player_data['url'] = player_field.split("href='")[1].split("'")[0]
            # Get the flag image URL
            parsed_player_data['flag_img_url'] = player_field.split("<img src='")[1].split("'")[0]
            # Extract the player's name
            parsed_player_data['name'] = player_field.split("loading='lazy'>")[1].split("<")[0]
        # Extract monetary values if the field contains 'accounting.formatMoney'
        elif "accounting.formatMoney" in player_field:
            # Get the field key and corresponding salary value
            field_key = player_field.split("'")[1].split("'")[0]
            value = player_field.split('"')[1].split('"')[0]
            parsed_player_data[field_key] = value
        # Extract position if 'position' is in the field
        elif "position" in player_field:
            parsed_player_data['position'] = player_field.split('"')[1].split('"')[0]
        # Extract age if 'age' is in the field
        elif "age" in player_field:
            parsed_player_data["age"] = player_field.split('"')[1].split('"')[0]
        # Extract country if 'country' is in the field
        elif "country" in player_field:
            parsed_player_data["country"] = player_field.split('"')[1].split('"')[0]

    # Append the parsed player data dictionary to the list of all players' data
    players_data.append(parsed_player_data)

# Display the final parsed data for the players
players_data

In [ ]:

df = pd.DataFrame(players_data)
df = df[["name", "weekly_gross_eur"]]

df['weekly_gross_eur'] = pd.to_numeric(df["weekly_gross_eur"], errors="coerce")

df = df.query("weekly_gross_eur > 0")
df.to_csv("premier_league_salaries.csv", index=False)

df